In [1]:
!python3 -m pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 119.5 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [2]:
!pip3 install arxiv
!pip3 install python-dotenv tiktoken
!pip3 install openai
#==0.27.8

  Obtaining dependency information for arxiv from https://files.pythonhosted.org/packages/f0/06/9b9d553d93e25ae27ec5ba794216afb1af248e43d85a35e922a85cbb396a/arxiv-1.4.8-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 198.6 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6066 sha256=18c17adca6ae1040d97949762302fa3eadb15a62ef2b215fb317ef38beee4b43
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Obtaining dependency information

In [3]:
import datetime as dt
import arxiv
import openai
import yaml


In [4]:
import openai

In [5]:
from dotenv import load_dotenv
import os

In [6]:
load_dotenv()

True

In [7]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [8]:
SYSTEM = """
### 指示 ###
論文の内容を理解した上で，重要なポイントを箇条書きで3点書いてください。

### 箇条書きの制約 ###
- 最大3個
- 日本語
- 箇条書き1個を50文字以内

### 対象とする論文の内容 ###
{text}

### 出力形式 ###
タイトル(和名)

- 箇条書き1
- 箇条書き2
- 箇条書き3
"""

In [9]:
# arXivの更新頻度を加味して、5日前の論文を検索
N_DAYS = 5
MAX_RESULT = 24  # 取得する論文数の上限
MODEL_NAME = "gpt-3.5-turbo-0613"
# MODEL_NAME = "gpt-3.5-turbo-instruct"
TEMPERATURE = 0.8

# テンプレートを用意
QUERY_TEMPLATE = '%28 ti:%22{}%22 OR abs:%22{}%22 %29 AND submittedDate: [{} TO {}]'

# 検索を行い、結果を取得する関数
def search_arxiv(keyword):
    # 2日前からN_DAYS前までの論文を検索
    today = dt.datetime.today() - dt.timedelta(days=2)
    base_date = today - dt.timedelta(days=N_DAYS)
    query = QUERY_TEMPLATE.format(keyword, keyword, base_date.strftime("%Y%m%d%H%M%S"), today.strftime("%Y%m%d%H%M%S"))

    search = arxiv.Search(
        query=query,
        max_results=MAX_RESULT,
        sort_by=arxiv.SortCriterion.SubmittedDate,
        sort_order=arxiv.SortOrder.Descending,
    )

    # results = [result for result in search.results() if set(result.categories) & CATEGORIES]
    results = []
    for result in search.results():
        # カテゴリーチェック
        # if not set(result.categories) & CATEGORIES:
        #     continue
        # 要約内でのキーワードの存在チェック
        if keyword.lower() in result.summary.lower():
            results.append(result)
    
    return results

# 論文の要約を取得する関数
def get_summary(result):
    text = f"title: {result.title}\nbody: {result.summary}"

    messages = [
        {"role" : "system", "content" : SYSTEM},
        {"role": "user", "content": text}
    ]
    
    response = openai.ChatCompletion.create(
        model=MODEL_NAME,
        messages=messages,
        temperature=TEMPERATURE,
    )
    
    return response["choices"][0]["message"]["content"]

In [10]:
# CATEGORIES = {
#     "cs.AI",  # 例: コンピュータサイエンスのAI分野
#     # "stat.ML",  # 例: 統計学の機械学習分野
#     # 必要に応じて他のカテゴリーを追加
# }

keyword = "LLM"
results = search_arxiv(keyword)

for result in results:
    summary = get_summary(result)
    print(f"title: {result.title}")
    print(f"published: {result.published}")
    print(f"abstruct: {result.summary}")
    print(f"PDFリンク: {result.pdf_url}")
    print(f"summary: {summary}")
    print("-" * 50)


title: uTalk: Bridging the Gap Between Humans and AI
published: 2023-10-04 11:30:13+00:00
abstruct: Large Language Models (LLMs) have revolutionized various industries by
harnessing their power to improve productivity and facilitate learning across
different fields. One intriguing application involves combining LLMs with
visual models to create a novel approach to Human-Computer Interaction. The
core idea behind this system is to develop an interactive platform that allows
the general public to leverage the capabilities of ChatGPT in their daily
lives. This is achieved by integrating several technologies such as Whisper,
ChatGPT, Microsoft Speech Services, and the state-of-the-art (SOTA) talking
head system, SadTalker, resulting in uTalk, an intelligent AI system. Users
will be able to converse with this portrait, receiving answers to whatever
questions they have in mind. Additionally, they could use uTalk for content
generation by providing an input and their image. This system is hos